In [5]:
import openai as client
from openai.types.beta import FunctionToolParam, AssistantToolParam
from openai.types.shared_params import FunctionDefinition
from app.agent import DuckDuckGoSearchTool

def search_duckduckgo(query: str) -> str:
    return DuckDuckGoSearchTool()._run(query)


functions: list[AssistantToolParam] = [
    FunctionToolParam(
        type="function",
        function=FunctionDefinition(
            name="search_duckduckgo",
            description="Search the web for information",
            parameters={
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query to search for",
                    }
                },
                "required": ["query"],
            },
        ),
    )
]


assistant = client.beta.assistants.create(
    name="Search Assistant",
    description="Search the web for information",
    model="gpt-3.5-turbo-1106",
    tools=functions,
)